In [2]:
import pandas as pd
import folium
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter
import matplotlib.pyplot as plt

def creer_carte_accidents(fichier_csv, nom_carte_html, centre=[44.8378, -0.5792], zoom=12):
    df = pd.read_csv(fichier_csv)

    # Conversion lat/long
    df["lat"] = df["lat"].astype(str).str.replace(",", ".")
    df["long"] = df["long"].astype(str).str.replace(",", ".")
    df["lat"] = pd.to_numeric(df["lat"], errors="coerce")
    df["long"] = pd.to_numeric(df["long"], errors="coerce")

    # Détection des lignes à géocoder
    a_geocoder = df[(df["lat"] == 0.0) | (df["long"] == 0.0)]

    if not a_geocoder.empty:
        print(f"⏳ Géocodage de {len(a_geocoder)} adresses manquantes...")

        geolocator = Nominatim(user_agent="geo_lukas")
        geocode = RateLimiter(geolocator.geocode, min_delay_seconds=1)

        for idx, row in a_geocoder.iterrows():
            if pd.notnull(row["adr"]):
                try:
                    location = geocode(f"{row['adr']}, Bordeaux, France")
                    if location:
                        df.at[idx, "lat"] = location.latitude
                        df.at[idx, "long"] = location.longitude
                        print(f"✅ {row['adr']} → ({location.latitude}, {location.longitude})")
                except Exception as e:
                    print(f"❌ Erreur géocodage {row['adr']} : {e}")

    # Filtrer coordonnées plausibles autour de Bordeaux
    df = df[
        (df["lat"].between(44.5, 45.5)) &
        (df["long"].between(-1.0, 0.0))
    ]

    print(f"Nombre d'accidents avec coordonnées valides : {len(df)}")

    if df.empty:
        print("⚠️ Aucune coordonnée valide trouvée.")
        return None

    # Création de la carte
    carte = folium.Map(location=centre, zoom_start=zoom, tiles='OpenStreetMap')

    for _, row in df.iterrows():
        folium.CircleMarker(
            location=[row["lat"], row["long"]],
            radius=3,
            color='red',
            fill=True,
            fill_opacity=0.7,
            popup=f"{row['Num_Acc']} - {row['adr']}"
        ).add_to(carte)

    carte.save(nom_carte_html)
    print(f"Carte enregistrée : {nom_carte_html}")
    return carte


In [3]:
creer_carte_accidents("accidents_bordeaux_cub_2023.csv", "carte_bordeaux_2023.html")

Nombre d'accidents avec coordonnées valides : 1359
Carte enregistrée : carte_bordeaux_2023.html
